In [13]:
import re
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup


class Tabelog:
    """
    食べログスクレイピングクラス
    test_mode=Trueで動作させると、最初のページの３店舗のデータのみを取得できる
    """
    def __init__(self, base_url, category, test_mode=False, p_ward='東京都内', begin_page=1, end_page=1, do_save_everytime=False):

        # 変数宣言
        self.store_id = ''
        self.store_id_num = 0
        self.store_name = ''
        self.score = 0
        self.ward = p_ward
        self.review_cnt = 0
        self.review = ''
        self.user_info = ''
        self.columns = ['store_id', 'store_name', 'score', 'ward', 'review_cnt', 'review', 'user_info']
        self.df = pd.DataFrame(columns=self.columns)
        self.__regexcomp = re.compile(r'\n|\s') # \nは改行、\sは空白
        self.do_save_everytime = do_save_everytime
        self.count_to_save = 0
        self.category = category

        page_num = begin_page # 店舗一覧ページ番号

        if test_mode:
            list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
            self.scrape_list(list_url, mode=test_mode)
        else:
            while True:
                list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
                if self.scrape_list(list_url, mode=test_mode) != True:
                    break

                # INパラメータまでのページ数データを取得する
                if page_num >= end_page:
                    break
                page_num += 1
        return

    def scrape_list(self, list_url, mode):
        """
        店舗一覧ページのパーシング
        """
        r = requests.get(list_url)
        if r.status_code != requests.codes.ok:
            return False

        soup = BeautifulSoup(r.content, 'html.parser')
        soup_a_list = soup.find_all('a', class_='list-rst__rst-name-target') # 店名一覧

        if len(soup_a_list) == 0:
            return False

        if mode:
            for soup_a in soup_a_list[:2]:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)
        else:
            for soup_a in soup_a_list:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)

        return True

    def scrape_item(self, item_url, mode):
        """
        個別店舗情報ページのパーシング
        """
        start = time.time()

        r = requests.get(item_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ item_url }')
            return

        soup = BeautifulSoup(r.content, 'html.parser')

        # 店舗名称取得
        # <h2 class="display-name">
        #     <span>
        #         麺匠　竹虎 新宿店
        #     </span>
        # </h2>
        store_name_tag = soup.find('h2', class_='display-name')
        store_name = store_name_tag.span.string
        print('{}→店名：{}'.format(self.store_id_num, store_name.strip()), end='')
        self.store_name = store_name.strip()

        # ラーメン屋、つけ麺屋以外の店舗は除外
        store_head = soup.find('div', class_='rdheader-subinfo') # 店舗情報のヘッダー枠データ取得
        store_head_list = store_head.find_all('dl')
        store_head_list = store_head_list[1].find_all('span')
        #print('ターゲット：', store_head_list[0].text)

        # if store_head_list[0].text not in {'ラーメン', 'つけ麺'}:
        #     print('ラーメンorつけ麺のお店ではないので処理対象外')
        #     self.store_id_num -= 1
        #     return

        # 評価点数取得
        #<b class="c-rating__val rdheader-rating__score-val" rel="v:rating">
        #    <span class="rdheader-rating__score-val-dtl">3.58</span>
        #</b>
        rating_score_tag = soup.find('b', class_='c-rating__val')
        rating_score = rating_score_tag.span.string
        print('  評価点数：{}点'.format(rating_score), end='')
        self.score = rating_score

        # 評価点数が存在しない店舗は除外
        if rating_score == '-':
            print('  評価がないため処理対象外')
            self.store_id_num -= 1
            return
       # 評価が3.5未満店舗は除外
    #     if float(rating_score) < 3.5:
    #         print('  食べログ評価が3.5未満のため処理対象外')
    #         self.store_id_num -= 1
    #         return

        # レビュー一覧URL取得
        #<a class="mainnavi" href="https://tabelog.com/tokyo/A1304/A130401/13143442/dtlrvwlst/"><span>口コミ</span><span class="rstdtl-navi__total-count"><em>60</em></span></a>
        review_tag_id = soup.find('li', id="rdnavi-review")
        review_tag = review_tag_id.a.get('href')

        # レビュー件数取得
        print('  レビュー件数：{}'.format(review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string), end='')
        self.review_cnt = review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string

        # レビュー一覧ページ番号
        page_num = 1 #1ページ*20 = 20レビュー 。この数字を変えて取得するレビュー数を調整。

        # レビュー一覧ページから個別レビューページを読み込み、パーシング
        # 店舗の全レビューを取得すると、食べログの評価ごとにデータ件数の濃淡が発生してしまうため、
        # 取得するレビュー数は１ページ分としている（件数としては１ページ*20=２0レビュー）
        while True:
            review_url = review_tag + 'COND-0/smp1/?lc=0&rvw_part=all&PG=' + str(page_num)
            #print('\t口コミ一覧リンク：{}'.format(review_url))
            print(' . ' , end='') #LOG
            if self.scrape_review(review_url) != True:
                break
            if page_num >= 1:
                break
            page_num += 1

        process_time = time.time() - start
        print('  取得時間：{}'.format(process_time))

        return

    def scrape_review(self, review_url):
        """
        レビュー一覧ページのパーシング
        """
        r = requests.get(review_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_url }')
            return False

        # 各個人の口コミページ詳細へのリンクを取得する
        #<div class="rvw-item js-rvw-item-clickable-area" data-detail-url="/tokyo/A1304/A130401/13141542/dtlrvwlst/B408082636/?use_type=0&amp;smp=1">
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review_url_list = soup.find_all('div', class_='rvw-item') # 口コミ詳細ページURL一覧

        if len(review_url_list) == 0:
            return False

        for url in review_url_list:
            review_detail_url = 'https://tabelog.com' + url.get('data-detail-url')
            #print('\t口コミURL：', review_detail_url)

            # 口コミのテキストを取得
            self.get_review_text(review_detail_url)

        return True

    def get_review_text(self, review_detail_url):
        """
        口コミ詳細ページをパーシング
        """
        r = requests.get(review_detail_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_detail_url }')
            return

        # ２回以上来訪してコメントしているユーザは最新の1件のみを採用
        #<div class="rvw-item__rvw-comment" property="v:description">
        #  <p>
        #    <br>すごい煮干しラーメン凪 新宿ゴールデン街本館<br>スーパーゴールデン1600円（20食限定）を喰らう<br>大盛り無料です<br>スーパーゴールデンは、新宿ゴールデン街にちなんで、ココ本店だけの特別メニューだそうです<br>相方と歌舞伎町のtohoシネマズの映画館でドラゴンボール超ブロリー を観てきた<br>ブロリー 強すぎるね(^^)面白かったです<br>凪の煮干しラーメンも激ウマ<br>いったん麺ちゅるちゅる感に、レアチャーと大トロチャーシューのトロけ具合もうめえ<br>煮干しスープもさすが！と言うほど完成度が高い<br>さすが食べログラーメン百名店<br>と言うか<br>2日連チャンで、近場の食べログラーメン百名店のうちの2店舗、昨日の中華そば葉山さんと今日の凪<br>静岡では考えられん笑笑<br>ごちそうさまでした
        #  </p>
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review = soup.find_all('div', class_='rvw-item__rvw-comment')#reviewが含まれているタグの中身をすべて取得
        if len(review) == 0:
            review = ''
        else:
            review = review[0].p.text.strip() # strip()は改行コードを除外する関数

        #print('\t\t口コミテキスト：', review)
        self.review = review
        
        info = soup.find_all('span', class_='rvw-item__rvwr-profile')
        if len(info) == 0:
            info = ''
        else:
            info = info[0].text

        self.user_info = info

        # データフレームの生成
        self.make_df()
        return

    def make_df(self):
        self.count_to_save += 1

        self.store_id = str(self.store_id_num).zfill(8) #0パディング
        se = pd.Series([self.store_id, self.store_name, self.score, self.ward, self.review_cnt, self.review, self.user_info], self.columns) # 行を作成
        self.df = self.df.append(se, self.columns) # データフレームに行を追加
        if self.do_save_everytime and self.count_to_save > 10:
            self.df.to_csv(f'temp_{self.category}.csv')
            self.count_to_save = 0
        return


In [7]:
category = 'italian'
tokyo_ramen_review = Tabelog(base_url="https://tabelog.com/rstLst/italian/",test_mode=False, p_ward='東京都内',end_page=10, do_save_everytime=True, category=category)
tokyo_ramen_review.df.to_csv(f"tokyo_{category}_review.csv")

1→店名：ペレグリーノ  評価点数：4.69点  レビュー件数：93 .   取得時間：28.70814871788025
2→店名：やまぐち  評価点数：4.62点  レビュー件数：130 .   取得時間：21.902745485305786
3→店名：長谷川 稔  評価点数：4.61点  レビュー件数：137 .   取得時間：23.422059059143066
4→店名：フォリオリーナ・デッラ・ポルタ・フォルトゥーナ  評価点数：4.54点  レビュー件数：78 .   取得時間：23.405022859573364
5→店名：TACUBO  評価点数：4.52点  レビュー件数：276 .   取得時間：31.012141704559326
6→店名：イル アオヤマ  評価点数：4.39点  レビュー件数：65 .   取得時間：24.044100284576416
7→店名：ウシマル  評価点数：4.37点  レビュー件数：203 .   取得時間：24.664801120758057
8→店名：トムクリオーザ  評価点数：4.30点  レビュー件数：121 .   取得時間：21.169634342193604
9→店名：ヴィラ・アイーダ  評価点数：4.30点  レビュー件数：121 .   取得時間：21.65917944908142
10→店名：プレゼンテ スギ  評価点数：4.28点  レビュー件数：94 .   取得時間：26.030564785003662
11→店名：クッチーナ イタリアーナ ガッルーラ  評価点数：4.27点  レビュー件数：264 .   取得時間：24.350899934768677
12→店名：K+  評価点数：4.25点  レビュー件数：138 .   取得時間：20.281397819519043
13→店名：RODEO  評価点数：4.21点  レビュー件数：167 .   取得時間：29.538203954696655
14→店名：オルト  評価点数：4.19点  レビュー件数：154 .   取得時間：23.923803091049194
15→店名：クイントカント  評価点数：4.16点  レビュー件数：313 .   取得時間：25.181528091430664
16→店名：pesceco  評価

121→店名：アンティカ・オステリア・デル・ポンテ  評価点数：3.77点  レビュー件数：240 .   取得時間：22.19605827331543
122→店名：ベントエマーレ  評価点数：3.77点  レビュー件数：218 .   取得時間：22.96811842918396
123→店名：リストランテ・ホンダ  評価点数：3.77点  レビュー件数：195 .   取得時間：21.475650310516357
124→店名：トルナヴェント  評価点数：3.77点  レビュー件数：154 .   取得時間：19.397703409194946
125→店名：ピッツェリアブラチェリア チェザリ  評価点数：3.76点  レビュー件数：556 .   取得時間：23.907231330871582
126→店名：トラットリア・築地パラディーゾ  評価点数：3.76点  レビュー件数：415 .   取得時間：23.892178297042847
127→店名：イル・ボッカローネ  評価点数：3.76点  レビュー件数：338 .   取得時間：24.040608644485474
128→店名：ラ・テンダ・ロッサ  評価点数：3.76点  レビュー件数：326 .   取得時間：30.138452768325806
129→店名：イタリア料理 フィオレンツァ  評価点数：3.76点  レビュー件数：312 .   取得時間：22.091819524765015
130→店名：ピッツェリア イル・タンブレッロ  評価点数：3.76点  レビュー件数：291 .   取得時間：22.932509183883667
131→店名：ダルマット 西麻布本店  評価点数：3.76点  レビュー件数：276 .   取得時間：22.29769730567932
132→店名：ダルブリガンテ  評価点数：3.76点  レビュー件数：272 .   取得時間：22.433863639831543
133→店名：アロマクラシコ  評価点数：3.76点  レビュー件数：269 .   取得時間：21.570698022842407
134→店名：ピャチェーレ  評価点数：3.76点  レビュー件数：245 .   取得時間：24.028319120407104
135→店名：代官山

In [ ]:
categories = ('yakiniku', 'izakaya', 'ramen', 'cafe', 'washoku', 'japanese', 'yakitori', 'yoshoku', 'french', 'spain', 'steak', 'hamburger', 'chinese', 'korea', 'thai', 'curry', 'viking', 'sweets', 'pan')
for category in categories:
    tokyo_ramen_review = Tabelog(base_url=f"https://tabelog.com/rstLst/{category}/", test_mode=False, p_ward='東京都内',end_page=3, do_save_everytime=True, category=category)
    tokyo_ramen_review.df.to_csv(f"tokyo_{category}_review.csv")    

1→店名：蕃 YORONIKU  評価点数：4.49点  レビュー件数：347 .   取得時間：29.27269196510315
2→店名：焼肉 ジャンボ はなれ  評価点数：4.48点  レビュー件数：317 .   取得時間：28.93094801902771
3→店名：赤坂 らいもん  評価点数：4.46点  レビュー件数：134 .   取得時間：25.984110116958618
4→店名：肉屋 雪月花 NAGOYA  評価点数：4.44点  レビュー件数：150 .   取得時間：24.642456531524658
5→店名：SATOブリアン 本店  評価点数：4.41点  レビュー件数：365 .   取得時間：31.946526527404785
6→店名：炭焼 金竜山  評価点数：4.36点  レビュー件数：431 .   取得時間：33.2867693901062
7→店名：肉割烹 上  評価点数：4.34点  レビュー件数：138 .   取得時間：25.563125610351562
8→店名：生ホルモン処 おさむちゃん。  評価点数：4.34点  レビュー件数：91 .   取得時間：25.660032510757446
9→店名：よろにく  評価点数：4.30点  レビュー件数：899 .   取得時間：32.089574337005615
10→店名：おにく 花柳  評価点数：4.29点  レビュー件数：256 .   取得時間：25.241140127182007
11→店名：スタミナ苑  評価点数：4.27点  レビュー件数：811 .   取得時間：33.59586548805237
12→店名：炭火焼 ゆうじ  評価点数：4.22点  レビュー件数：596 .   取得時間：29.63309144973755
13→店名：炭火焼肉 なかはら  評価点数：4.20点  レビュー件数：251 .   取得時間：24.729472875595093
14→店名：サトブリDA  評価点数：4.15点  レビュー件数：87 .   取得時間：18.992507696151733
15→店名：SATOブリアン にごう  評価点数：4.14点  レビュー件数：356 .   取得時間：28.305601835250854
16→店名：

6→店名：メンドコロ キナリ  評価点数：3.98点  レビュー件数：282 .   取得時間：21.237658977508545
7→店名：宍道湖しじみ中華蕎麦 琥珀  評価点数：3.97点  レビュー件数：198 .   取得時間：20.898286819458008
8→店名：自家製手もみ麺 鈴ノ木  評価点数：3.97点  レビュー件数：126 .   取得時間：20.185933351516724
9→店名：麺尊 RAGE  評価点数：3.96点  レビュー件数：613 .   取得時間：29.581578254699707
10→店名：かしわぎ  評価点数：3.95点  レビュー件数：396 .   取得時間：23.10732078552246
11→店名：麺屋 さくら井  評価点数：3.95点  レビュー件数：377 .   取得時間：28.73104691505432
12→店名：中華蕎麦にし乃  評価点数：3.94点  レビュー件数：500 .   取得時間：24.14920687675476
13→店名：らーめん改  評価点数：3.94点  レビュー件数：491 .   取得時間：22.439584493637085
14→店名：純手打ち だるま  評価点数：3.94点  レビュー件数：215 .   取得時間：19.895549535751343
15→店名：中華そば しば田  評価点数：3.93点  レビュー件数：737 .   取得時間：24.83658242225647
16→店名：らぁ麺やまぐち  評価点数：3.93点  レビュー件数：348 .   取得時間：23.83180594444275
17→店名：らーめんMAIKAGURA  評価点数：3.93点  レビュー件数：201 .   取得時間：19.16059136390686
18→店名：中華ソバ 伊吹  評価点数：3.92点  レビュー件数：472 .   取得時間：24.095057249069214
19→店名：Ramen　にじゅうぶんのいち  評価点数：3.92点  レビュー件数：243 .   取得時間：21.63202929496765
20→店名：支那蕎麦屋 藤花  評価点数：3.92点  レビュー件数：193 .   取得時間：20.8587534427642

6→店名：緒方  評価点数：4.74点  レビュー件数：240 .   取得時間：33.087305307388306
7→店名：天寿し 京町店  評価点数：4.73点  レビュー件数：365 .   取得時間：34.3472363948822
8→店名：松川  評価点数：4.72点  レビュー件数：230 .   取得時間：34.25602436065674
9→店名：東麻布 天本  評価点数：4.71点  レビュー件数：219 .   取得時間：35.20490789413452
10→店名：飯田  評価点数：4.71点  レビュー件数：96 .   取得時間：22.697532176971436
11→店名：鮨 なんば 日比谷  評価点数：4.70点  レビュー件数：227 .   取得時間：32.15432357788086
12→店名：新ばし 星野  評価点数：4.68点  レビュー件数：73 .   取得時間：26.493895053863525
13→店名：鮨 さかい  評価点数：4.66点  レビュー件数：162 .   取得時間：32.4769446849823
14→店名：本湖月  評価点数：4.65点  レビュー件数：156 .   取得時間：24.638127088546753
15→店名：富小路 やま岸  評価点数：4.65点  レビュー件数：154 .   取得時間：30.968095779418945
16→店名：柳家  評価点数：4.64点  レビュー件数：407 .   取得時間：32.19532370567322
17→店名：日本料理 たかむら  評価点数：4.64点  レビュー件数：154 .   取得時間：22.885454893112183
18→店名：鳥しき  評価点数：4.63点  レビュー件数：663 .   取得時間：40.0787889957428
19→店名：比良山荘  評価点数：4.63点  レビュー件数：259 .   取得時間：23.08658719062805
20→店名：野嵯和  評価点数：4.62点  レビュー件数：84 .   取得時間：23.84844398498535
21→店名：すし処 めくみ  評価点数：4.61点  レビュー件数：176 .   取得時間：24.38992524147033

17→店名：焼鳥 市松  評価点数：3.96点  レビュー件数：281 .   取得時間：25.182390928268433
18→店名：里葉亭  評価点数：3.95点  レビュー件数：352 .   取得時間：23.82234501838684
19→店名：鳥かど  評価点数：3.92点  レビュー件数：208 .   取得時間：22.760953187942505
20→店名：鳥長  評価点数：3.90点  レビュー件数：197 .   取得時間：24.562852144241333
21→店名：吾一  評価点数：3.89点  レビュー件数：44 .   取得時間：20.84152579307556
22→店名：焼鳥 みずき  評価点数：3.87点  レビュー件数：100 .   取得時間：24.44910430908203
23→店名：蘭奢待  評価点数：3.85点  レビュー件数：158 .   取得時間：23.347010612487793
24→店名：焼鳥 谷口  評価点数：3.85点  レビュー件数：43 .   取得時間：21.783585786819458
25→店名：鳥おか  評価点数：3.84点  レビュー件数：129 .   取得時間：20.90071153640747
26→店名：焼鶏 ひらこ  評価点数：3.84点  レビュー件数：65 .   取得時間：20.31512999534607
27→店名：せら  評価点数：3.84点  レビュー件数：46 .   取得時間：21.736713886260986
28→店名：焼鳥今井  評価点数：3.82点  レビュー件数：147 .   取得時間：22.681617498397827
29→店名：焼鳥 茜  評価点数：3.82点  レビュー件数：54 .   取得時間：21.87537145614624
30→店名：千亀  評価点数：3.81点  レビュー件数：233 .   取得時間：22.749901294708252
31→店名：焼鳥 うの  評価点数：3.81点  レビュー件数：88 .   取得時間：22.80646324157715
32→店名：焼き鳥 丈参  評価点数：3.80点  レビュー件数：250 .   取得時間：23.204519987106323
33→店名：鳥恵

24→店名：レストラン・モリエール  評価点数：4.25点  レビュー件数：225 .   取得時間：19.38603448867798
25→店名：レストラン ラ フィネス  評価点数：4.25点  レビュー件数：106 .   取得時間：27.572983503341675
26→店名：レ セゾン  評価点数：4.24点  レビュー件数：390 .   取得時間：27.123525619506836
27→店名：祇園 呂色  評価点数：4.24点  レビュー件数：124 .   取得時間：20.34160017967224
28→店名：フロリレージュ  評価点数：4.23点  レビュー件数：293 .   取得時間：26.987486600875854
29→店名：トゥールダルジャン 東京  評価点数：4.23点  レビュー件数：246 .   取得時間：21.553163766860962
30→店名：コート ドール  評価点数：4.22点  レビュー件数：454 .   取得時間：24.050206184387207
31→店名：北島亭  評価点数：4.20点  レビュー件数：297 .   取得時間：25.306641101837158
32→店名：モトイ  評価点数：4.20点  レビュー件数：126 .   取得時間：22.725270748138428
33→店名：オルト  評価点数：4.19点  レビュー件数：154 .   取得時間：22.5904438495636
34→店名：Ode  評価点数：4.18点  レビュー件数：180 .   取得時間：22.476642370224
35→店名：スガラボヴィー  評価点数：4.18点  レビュー件数：41 .   取得時間：24.88758134841919
36→店名：ラ・メール ザ クラシック  評価点数：4.17点  レビュー件数：181 .   取得時間：21.717849254608154
37→店名：レストラン クレッセント  評価点数：4.17点  レビュー件数：144 .   取得時間：23.738831043243408
38→店名：TTOAHISU  評価点数：4.17点  レビュー件数：78 .   取得時間：22.255497932434082
39→店名：クイントカント 